# Welcome to Astir
### A cell type assignment algorithm for single cell multiplexed imaging and proteomic data.

This is a reproducible notebook with some sample data for you to explore Astir's capability.

In [ ]:
import os
import json
import yaml
import argparse

import scanpy
import anndata as ad
import pandas as pd
import numpy as np

from itertools import chain
from sklearn.metrics import adjusted_rand_score

from astir.data import from_anndata_yaml

In [ ]:
INTEGRATION_METHOD = 'exprs'
DATASET = 'IMMUcan_2022_CancerExample'
EXPERIMENT_DIR = f"/home/dani/Documents/Thesis/Methods/IMCBenchmark/output/{DATASET}/astir/{INTEGRATION_METHOD}"
CONFIG_PATH = os.path.join(EXPERIMENT_DIR, 'config.json')

# load the params
with open(CONFIG_PATH) as f:
    config = json.load(f)

In [ ]:
parser = argparse.ArgumentParser(description='astir')

args = parser.parse_args(args=[])
args.dataset = config['dataset']
args.markers = config['markers']
args.threshold = config['results_threshold']
args.max_epochs = config['max_epochs']
args.learning_rate = config['learning_rate']
args.batch_size = config['batch_size']
args.delta_loss = config['delta_loss']
args.n_init = config['n_init']
args.n_init_epochs = config['n_init_epochs']
args.seed = config['seed']

In [ ]:
args.threshold = 0.5

In [ ]:
args

## Load data

In [ ]:
adata = ad.read_h5ad(args.dataset)

adata.obs

Load marker file for each cell type of interest

In [ ]:
with open(args.markers, 'r') as file:
    markers_file = yaml.load(file, Loader=yaml.FullLoader)
    
print(markers_file['cell_types'])

## Creating an Astir object

Next, we need to convert the AnnData object into an Astir object. This also requires a list of marker files for each cell type of interest.





In [ ]:
astir = from_anndata_yaml(
    anndata_file = args.dataset, 
    marker_yaml = args.markers, 
    create_design_mat = True, 
    random_seed = args.seed
)

astir

## Training the model

Astir takes the following as input:


*   Batch size
*   Maximum number of epochs
*   A learning rate
*   An initial set of epochs




To assign cell types simply call `fit_type` as follows.

In [ ]:
astir.fit_type(
    max_epochs = args.max_epochs,
    batch_size = args.batch_size,
    learning_rate = args.learning_rate,
    delta_loss = args.delta_loss,
    n_init = args.n_init,
    n_init_epochs = args.n_init_epochs
)

Importantly, Astir automatically creates a design matrix based on on additional covariates in the data such as batches. This is done using additional columns already present in the input data. In the example data of this vignette we have included a 'batch' column for the purposes of illustration.

We can get the cell type assignments in one of two ways:
1. Using `get_celltypes()`: this returns the most likely cell type or classifies a cell as unknown if no cell type has a probability above 0.7. (This threshold can be altered by the user with the threshold argument)
2. Using `get_celltype_probabilities()`: this returns the probabilty of each cell being assigned to any given cell type.

In [ ]:
astir.get_celltypes(threshold=args.threshold).value_counts()

In [ ]:
astir.get_celltype_probabilities()

## Post-fit diagnostics

As a sanity check for the user, Astir also outputs a set of post-fit diagnostics. These ensure that all cell types express their marker proteins at significantly higher levels that other cells.


In [ ]:
astir.diagnostics_celltype()

## Analyzing the results

Now that we have assigned cell types to all cells, we can do some basic analysis.

In [ ]:
adata.obs['pred'] = astir.get_celltypes(threshold=0.7).rename(columns={'cell_type': 'pred'})
adata.obs['pred_prob'] = astir.get_celltype_probabilities().apply(lambda row: np.max(row), axis=1)
adata.obs['prob_list'] = astir.get_celltype_probabilities().apply(lambda row: row.tolist(), axis=1)

results_df = adata.obs[['batch', 'cell_id', 'cell_type', 'pred', 'pred_prob', 'prob_list']]
results_df = results_df.rename(columns={'batch': 'image_id', 'cell_type': 'label'})

results_df.to_csv(os.path.join(EXPERIMENT_DIR, 'astir_results.csv'), index=False)

results_df

Let us start with a UMAP plot coloured by cell type. (This may take a while because it has to run UMAP first).

In [ ]:
scanpy.pp.neighbors(adata)
scanpy.tl.umap(adata)
scanpy.pl.umap(adata, color = ['cell_type', 'pred'], size = 14, ncols = 3, wspace = 0.3)

In [ ]:
markers = list(set(chain.from_iterable([x for x in markers_file['cell_types'].values()])))

scanpy.pl.heatmap(adata, markers, groupby='pred', swap_axes=True, standard_scale='var')

Calculate ARI score compared to ground truth labels

In [ ]:
ari_score = adjusted_rand_score(results_df['label'], results_df['pred'])

print("Adjusted Rand Index Score:", ari_score)